In [1]:
!pip install peft datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, TaskType
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

In [3]:
from datasets import Dataset

In [4]:
# Creating a list of lists with reviews and associated sentiments
dataset = [
    ["The SR Watch exceeded my expectations. It's stylish and functional.", "Positive"],
    ["I'm very disappointed with the SR Watch. It stopped working after just a few days.", "Negative"],
    ["The SR Watch is a decent product for its price. It's nothing extraordinary, but it gets the job done.", "Neutral"],
    ["I absolutely love my SR Watch! It's sleek, comfortable, and works flawlessly.", "Positive"],
    ["The SR Watch has a lot of features, but it's quite complicated to use. Not recommended for beginners.", "Negative"],
    ["I'm satisfied with my SR Watch purchase. It looks elegant and performs well.", "Positive"],
    ["The SR Watch is okay, but it could use some improvements in terms of durability.", "Neutral"],
    ["This is the best smartwatch I've ever owned! The SR Watch has exceeded all my expectations.", "Positive"],
    ["The SR Watch is a good value for the money. It's durable and has a long battery life.", "Positive"],
    ["I'm on the fence about the SR Watch. It has some good features, but the design is not my favorite.", "Mixed"],
    ["I regret buying the SR Watch. It's not user-friendly and the battery life is disappointing.", "Negative"],
    ["The SR Watch looks stylish, but it's not very durable. Mine broke after just a few weeks of use.", "Negative"],
    ["I'm quite impressed with the SR Watch. It's accurate, comfortable to wear, and has a sleek design.", "Positive"],
    ["The SR Watch is average. It doesn't stand out from other smartwatches on the market.", "Neutral"],
    ["The SR Watch is a great companion for my workouts. It tracks my activity accurately and motivates me to stay active.", "Positive"],
    ["I'm not sure what to think about the SR Watch. It has some good features, but it's not perfect.", "Mixed"],
    ["I had high hopes for the SR Watch, but it failed to meet my expectations. The performance is subpar.", "Negative"],
    ["The SR Watch is perfect for everyday use. It's lightweight, comfortable, and has a long-lasting battery.", "Positive"],
    ["The SR Watch is overpriced for what it offers. There are better options available in the market.", "Negative"],
    ["I'm still undecided about the SR Watch. It has its pros and cons.", "Mixed"],
    ["I've had a positive experience with the SR Watch so far. It's reliable and easy to use.", "Positive"],
    ["The SR Watch is a solid choice for fitness enthusiasts. It accurately tracks my workouts and helps me stay on top of my health goals.", "Positive"],
    ["I bought the SR Watch for its stylish design, but I'm disappointed with its performance. It frequently lags and freezes.", "Negative"],
    ["I've been using the SR Watch for a month now, and it's been a game-changer for me. It's intuitive to use and has improved my productivity.", "Positive"],
    ["The SR Watch is a bit pricey, but it's worth every penny. The build quality is exceptional, and it has all the features I need.", "Positive"],
    ["I'm underwhelmed by the SR Watch. It feels cheaply made, and the battery life is shorter than expected.", "Negative"],
    ["The SR Watch is a versatile device. It's equally suitable for casual everyday wear and intense workouts.", "Positive"],
    ["I had high hopes for the SR Watch, but it failed to live up to expectations. The heart rate monitor is inaccurate, and the software is buggy.", "Negative"],
    ["I'm pleasantly surprised by the SR Watch. It's comfortable to wear all day, and the battery life is impressive.", "Positive"],
    ["The SR Watch is a mixed bag. While it has some innovative features, it falls short in terms of reliability and durability.", "Mixed"],
    ["I've tried several smartwatches, but the SR Watch stands out for its elegant design and user-friendly interface.", "Positive"],
    ["The SR Watch is not for everyone. It's geared towards fitness enthusiasts and may not appeal to those looking for a traditional timepiece.", "Neutral"],
    ["I'm on the fence about the SR Watch. It has some great features, but the software needs improvement.", "Mixed"],
    ["The SR Watch is a disappointment. It feels cheaply made, and the screen scratches easily.", "Negative"],
    ["I'm blown away by the SR Watch! It's sleek, stylish, and packed with features. Definitely worth the investment.", "Positive"],
    ["The SR Watch is a decent entry-level smartwatch. It has basic functionality and a budget-friendly price tag.", "Neutral"],
    ["I'm skeptical about the SR Watch. The reviews are mixed, and I'm not sure if it's worth the hype.", "Mixed"],
    ["I've had a positive experience with the SR Watch customer service team. They were prompt in addressing my concerns.", "Positive"],
    ["The SR Watch is a letdown. The step counter is inaccurate, and the display is prone to glare.", "Negative"],
    ["I'm impressed by the SR Watch's battery life. It lasts for days on a single charge, even with continuous use.", "Positive"],
    ["The SR Watch looks sleek, but it lacks some basic features like GPS. Disappointed with the limited functionality.", "Negative"],
    ["I'm thoroughly impressed by the SR Watch's performance. It's reliable, accurate, and comfortable to wear.", "Positive"],
    ["The SR Watch is a versatile companion for outdoor activities. Its rugged design and water resistance make it perfect for adventure seekers.", "Positive"],
    ["I had high hopes for the SR Watch, but it fell short of expectations. The battery life is disappointing, and the software is buggy.", "Negative"],
    ["The SR Watch is a stylish accessory, but it lacks some advanced features found in other smartwatches.", "Neutral"],
    ["I'm satisfied with the SR Watch's performance. It's intuitive to use, and the battery lasts throughout the day.", "Positive"],
    ["The SR Watch is a good budget option for those looking for basic smartwatch features. Don't expect anything fancy.", "Neutral"],
    ["I'm on the fence about the SR Watch. It has some great features, but the build quality feels a bit flimsy.", "Mixed"],
    ["I'm disappointed with the SR Watch's accuracy. It frequently miscounts steps and fails to track workouts properly.", "Negative"],
    ["The SR Watch is a great investment for anyone looking to improve their fitness routine. It provides valuable insights and motivation.", "Positive"],
    ["The SR Watch looks sleek, but it's not very comfortable to wear for long periods. The strap irritates my skin.", "Negative"],
    ["I'm amazed by the SR Watch's battery life. It lasts for days on a single charge, even with continuous use.", "Positive"],
    ["The SR Watch is a disappointment. It feels bulky on the wrist, and the screen is prone to scratches.", "Negative"],
    ["I'm impressed by the SR Watch's durability. It withstands daily wear and tear without any issues.", "Positive"],
    ["The SR Watch is a great value for the price. It offers all the essential features without breaking the bank.", "Positive"],
    ["I had high expectations for the SR Watch, but it failed to deliver. The interface is clunky, and the notifications are unreliable.", "Negative"],
    ["The SR Watch is perfect for fitness enthusiasts. It accurately tracks workouts and provides valuable insights.", "Positive"],
    ["I'm underwhelmed by the SR Watch. It's lacking in features compared to other smartwatches in the same price range.", "Negative"],
    ["The SR Watch is a game-changer for my fitness routine. It keeps me motivated and on track with my goals.", "Positive"],
    ["I'm disappointed with the SR Watch's build quality. It feels cheap and plasticky.", "Negative"],
    ["I'm pleasantly surprised by the SR Watch's performance. It's reliable and easy to use, even for beginners.", "Positive"],
    ["The SR Watch is a great addition to my daily routine. It's comfortable to wear and has a sleek design.", "Positive"],
    ["I'm disappointed with the SR Watch's battery life. It drains quickly, especially when using GPS.", "Negative"],
    ["The SR Watch is perfect for tracking my runs. It provides accurate data and helps me improve my performance.", "Positive"],
    ["I'm satisfied with the SR Watch's performance. It's reliable and easy to navigate.", "Positive"],
    ["The SR Watch is a stylish accessory, but it lacks some advanced features found in other smartwatches.", "Neutral"],
    ["I'm on the fence about the SR Watch. It has some great features, but the build quality feels a bit flimsy.", "Mixed"],
    ["The SR Watch exceeded my expectations. It's durable, functional, and looks great on my wrist.", "Positive"],
    ["I regret buying the SR Watch. It's not as user-friendly as advertised, and the customer service was unhelpful.", "Negative"],
    ["The SR Watch is a disappointment. It frequently disconnects from my phone, and the app crashes often.", "Negative"],
    ["I'm impressed by the SR Watch's accuracy. It tracks my workouts and sleep patterns with precision.", "Positive"],
    ["The SR Watch is a good value for the price. It offers all the essential features without breaking the bank.", "Positive"],
    ["I'm still undecided about the SR Watch. It has some promising features, but I'm not sure if it's worth the investment.", "Mixed"],
    ["The SR Watch is a letdown. The heart rate monitor is unreliable, and the notifications are delayed.", "Negative"],
    ["I'm blown away by the SR Watch's battery life. It lasts for days on a single charge, even with heavy usage.", "Positive"],
    ["The SR Watch is a versatile companion for outdoor activities. It's rugged, waterproof, and packed with useful features.", "Positive"],
    ["I'm underwhelmed by the SR Watch. It lacks customization options, and the interface feels outdated.", "Negative"],
    ["The SR Watch is perfect for fitness enthusiasts. It accurately tracks workouts and provides real-time feedback.", "Positive"],
    ["I'm disappointed with the SR Watch's build quality. The strap broke after just a few weeks of use.", "Negative"],
    ["The SR Watch is a game-changer for my daily routine. It keeps me organized and motivated throughout the day.", "Positive"],
    ["I had high hopes for the SR Watch, but it failed to impress. The touchscreen is unresponsive, and the software is buggy.", "Negative"],
    ["The SR Watch is a great addition to my daily routine. It's comfortable to wear and has a sleek design.", "Positive"],
    ["I'm disappointed with the SR Watch's battery life. It drains quickly, especially when using GPS.", "Negative"],
    ["The SR Watch is perfect for tracking my runs. It provides accurate data and helps me improve my performance.", "Positive"],
    ["I'm satisfied with the SR Watch's performance. It's reliable and easy to navigate.", "Positive"],
    ["The SR Watch is a stylish accessory, but it lacks some advanced features found in other smartwatches.", "Neutral"],
    ["I'm on the fence about the SR Watch. It has some great features, but the build quality feels a bit flimsy.", "Mixed"],
    ["The SR Watch exceeded my expectations. It's durable, functional, and looks great on my wrist.", "Positive"],
    ["I regret buying the SR Watch. It's not as user-friendly as advertised, and the customer service was unhelpful.", "Negative"],
    ["The SR Watch is a disappointment. It frequently disconnects from my phone, and the app crashes often.", "Negative"],
    ["I'm impressed by the SR Watch's accuracy. It tracks my workouts and sleep patterns with precision.", "Positive"],
    ["The SR Watch is a good value for the price. It offers all the essential features without breaking the bank.", "Positive"],
    ["I'm still undecided about the SR Watch. It has some promising features, but I'm not sure if it's worth the investment.", "Mixed"],
    ["The SR Watch is a letdown. The heart rate monitor is unreliable, and the notifications are delayed.", "Negative"],
    ["I'm blown away by the SR Watch's battery life. It lasts for days on a single charge, even with heavy usage.", "Positive"],
    ["The SR Watch is a versatile companion for outdoor activities. It's rugged, waterproof, and packed with useful features.", "Positive"],
    ["I'm underwhelmed by the SR Watch. It lacks customization options, and the interface feels outdated.", "Negative"],
    ["The SR Watch is perfect for fitness enthusiasts. It accurately tracks workouts and provides real-time feedback.", "Positive"],
    ["I'm disappointed with the SR Watch's build quality. The strap broke after just a few weeks of use.", "Negative"],
    ["The SR Watch is a game-changer for my daily routine. It keeps me organized and motivated throughout the day.", "Positive"],
    ["I had high hopes for the SR Watch, but it failed to impress. The touchscreen is unresponsive, and the software is buggy.", "Negative"],
    ["The SR Watch exceeded my expectations. It's sleek, lightweight, and has a long-lasting battery.", "Positive"],
    ["I'm frustrated with the SR Watch's performance. It frequently loses connection with my phone, making it unreliable.", "Negative"],
    ["The SR Watch is perfect for everyday wear. It's comfortable, stylish, and packed with useful features.", "Positive"],
    ["I'm disappointed with the SR Watch's durability. The screen scratches easily, and the band feels flimsy.", "Negative"],
    ["The SR Watch is a versatile device. It seamlessly integrates with my smartphone and helps me stay organized.", "Positive"],
    ["I'm underwhelmed by the SR Watch's battery life. It doesn't last as long as advertised, especially with GPS enabled.", "Negative"],
    ["The SR Watch is a reliable companion for my workouts. It accurately tracks my progress and keeps me motivated.", "Positive"],
    ["I'm disappointed with the SR Watch's performance. It frequently freezes, and the touchscreen is unresponsive.", "Negative"],
    ["The SR Watch is a great investment. It's durable, stylish, and packed with features.", "Positive"],
    ["I regret buying the SR Watch. It's not as user-friendly as advertised, and the battery life is disappointing.", "Negative"]
]

In [5]:
df = pd.DataFrame(dataset, columns=["Human", "Assistant"])

In [6]:
df.head()

,Human,Assistant
0,The SR Watch exceeded my expectations. It's st...,Positive
1,I'm very disappointed with the SR Watch. It st...,Negative
2,The SR Watch is a decent product for its price...,Neutral
3,"I absolutely love my SR Watch! It's sleek, com...",Positive
4,"The SR Watch has a lot of features, but it's q...",Negative


In [7]:
df["Human"] = df["Human"].apply(lambda x: "Human: " + x + " Assistant: ")

In [8]:
df["Human"][0]

"Human: The SR Watch exceeded my expectations. It's stylish and functional. Assistant: "

In [9]:
model_id = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
def preprocess_function(sample, padding="max_length"):
    model_inputs = tokenizer(sample["Human"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["Assistant"], max_length=256, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
train_data = Dataset.from_pandas(df)

In [12]:
train_data

Dataset({
    features: ['Human', 'Assistant'],
    num_rows: 111
})

In [13]:
train_tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

In [14]:
train_tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 111
})

In [15]:
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.1,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

In [16]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989059049678777


In [17]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
output_dir="lora-flan-t5-large-sentiment"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    push_to_hub = True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
)
model.config.use_cache = False

trainer.train()
peft_save_model_id="lora-flan-t5-large-sentiment"
trainer.model.save_pretrained(peft_save_model_id, push_to_hub=True)
tokenizer.save_pretrained(peft_save_model_id, push_to_hub=True)
trainer.model.base_model.save_pretrained(peft_save_model_id, push_to_hub=True)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
28,1.038900


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15G [00:00<?, ?B/s]

In [20]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "lora-flan-t5-large-sentiment"
config = PeftConfig.from_pretrained(peft_model_id)

model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0}).cuda()
model.eval()

sample = "Human: \nSR Watch is good to watch. \nAssistant: "
input_ids = tokenizer(sample, return_tensors="pt", truncation=True, max_length=256).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, do_sample=True, top_p=0.9, max_length=256)
print(f"{sample}")

print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])


Human: 
SR Watch is good to watch. 
Assistant: 
Positive


In [21]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
